In [1]:
import json
from src.data import DATA_DIR
from hazm import word_tokenize
from src.utils.io import read_file, read_tel_messages, remove_stopwords
from hazm import Normalizer, word_tokenize, sent_tokenize
from collections import Counter

In [2]:
stopwords = read_file(DATA_DIR/'persian_stop_words.txt').split('\n')

2022-03-19 13:18:42.081 | INFO     | src.utils.io:read_file:22 - Loding data...


In [3]:
with open(DATA_DIR / 'cs_stack.json') as fp:
    data = json.load(fp)

In [17]:
def search_in_sent(text: str=None, text_path: str=None,search_val: list=None) -> list:
    """
    :param text: A text containig multi lines
    :param text_path: Path of a text containig multi lines
    :param search_val: If one of these values exist in setence, that sentece will be returned
    """
    sentences = []
    if not text is None:
        text_ = text
    
    elif not text_path is None:
        with open(text_path) as fp:
            text_ = fp.read()
            
    else:
        return sentencs

    textlines = sent_tokenize(text_)
    for sen in textlines:
        if (sum([i in sen for i in search_val]) > 0) and (len(sen) >= 3):
            sentences.append(sen)                
        return sentences

In [22]:
data_text = []
for msg in data['messages']:
    if isinstance(msg['text'], str):
        msg_ = msg['text']
        
    elif isinstance(msg['text'][0], str):
        msg_ = msg['text'][0]
    
    else:
        pass
    
    msg_ = remove_stopwords(msg_, stopwords)
    if sum([i in msg_ for i in ['؟', '?']]) > 0:
        data_text.extend(search_in_sent(text=msg_, search_val=['؟', '?']))

In [21]:
Tokens = word_tokenize(" ".join(data_text))
Counter(Tokens).most_common(20)

[('؟', 2101),
 ('سلام', 650),
 ('دوستان', 482),
 ('،', 294),
 ('نصب', 265),
 ('ببخشید', 185),
 ('کردم', 149),
 ('ثبت', 146),
 ('کلاس', 124),
 ('جلسه', 113),
 ('کنیم', 106),
 ('مشکل', 102),
 ('کار', 102),
 ('چیه', 101),
 ('سوال', 100),
 ('؟؟', 100),
 ('دوره', 99),
 ('چی', 99),
 ('کرد', 97),
 ('بخیر', 94)]

In [70]:
question = list(filter(lambda text: ('?' in text) or ('؟' in text), data_text))
len(question)

2879

In [6]:
read_tel_messages(file_path=DATA_DIR / 'cs_stack.json', normalize=True)

['',
 'آقای حجازی \nاگر احیانا تعداد نفرات متقاضی شرکت در دوره بیش از ظرفیت پلتفرم\u200cهای برگزاری دوره بود، خواهشا دوره را به صورت آفلاین برگزار کنید\n\nما نمی\u200cخواهیم دوره رو از دست بدیم \n',
 'سلام به همه. \nدوستان پیش\u200cزمینه\u200cهای یادگیری پایتون برای کسی که کدنویسی نکرده کلا چیه؟ \nصرفا کلیدواژه می\u200cخوام که بتونم سرچ کنم. ممنونم ازتون',
 'جادی توی یوتیوبش یه پلی لیست داره به اسم درک برنامه نویسی\nاونو ببینید',
 'مرسی🙏',
 'موافقم\nمن حتی نتونستم داخل کلاس جوین بشم آفلاین هم برام باز نشد',
 'ممنونم. Jady؟',
 'Jadi',
 'Jadi',
 'داخل udemy یه سری انلاین کورسای رایگان هست تحت عنوان how to program فقط اگر با زبان انگلیسی راحتید به کار میاد',
 'تو سایت مکتبخونه هم برید ویدیو درک برنامه نویسی جادی موجوده',
 'ممنونم دوستان',
 'لینک پلی لیست درک برنامه نویسی از کانال جادی',
 'ببینید الگوریتم و فلوچارت تمرین کنید یه دید برنامه نویسی داشته باشید.',
 'بچه\u200cها نصب کردید اون ubuntu رو توی ویندوز \nمن نصب کردم اون مراحل یوزرنیم و پسورد هم رد شد و همه چی اکی \nولی وقتی خارج کردم